# Introduction

In this copy, I just quickly wanted to re-run something 

In [ ]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
from agent import Agent
%matplotlib inline

#Environment
env = gym.make('MountainCarContinuous-v0')
env.seed(1)  # for comparison
num_states = env.observation_space.shape[0]
num_actions = env.action_space.shape[0]

#Agent
lr,gamma = 0.001, 0.1
agent = Agent(num_states, num_actions, lr, gamma)
agent.memory_size = 10**4
agent.batchsize = 256
learning_start = 25*agent.batchsize
agent.tau = 0.01


#Train
EPISODES = 200
MAX_STEPS = 1000
scores = []
for e in range(1,EPISODES+1):
    state = env.reset()
    reward_sum = 0
    done = False
    steps = 0
    t1 = time.time()
    
    while not done:
        #env.render()
        state = np.reshape(state, [1, num_states])  #reshape for keras
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward_sum += reward
        agent.remember(state[0], action, reward, next_state, done)
        state = next_state
        
        if len(agent.memory) > learning_start:
            agent.train_models()
        
        steps += 1
        if done or steps > MAX_STEPS:
            break
    
    #Learn & print results
    scores.append(reward_sum)
    t2 = time.time()
    if e % 1 == 0:
        print '(episode, score, steps, T (mins)) = ' + str((e,reward_sum, steps, (t2-t1)/60.0))

plt.plot(scores)
#np.savetxt('stats/scores_inverted_pendulum.txt',scores)

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(episode, score, steps, T (mins)) = (1, -0.00040056172247518885, 999, 0.016531534989674888)
(episode, score, steps, T (mins)) = (2, -0.00039159806460931676, 999, 0.010755252838134766)
(episode, score, steps, T (mins)) = (3, -0.0003908788244822198, 999, 0.010851951440175374)
(episode, score, steps, T (mins)) = (4, -0.00039093382509105217, 999, 0.012474751472473145)
(episode, score, steps, T (mins)) = (5, -0.00039909349371099775, 999, 0.012235514322916667)
(episode, score, steps, T (mins)) = (6, -0.0003935323635723678, 999, 0.014521296819051106)
(episode, score, steps, T (mins)) = (7, -44.35271829480654, 999, 2.0074299693107607)
(episode, score, steps, T (mins)) = (8, -99.88905611089781, 999, 3.3815884987513223)
(episode, score, steps, T (mins)) = (9, -99.8997198471804, 999, 3.4354946851730346)


### Watch a smart agent

In [4]:
done = False
while not done:
    env.render()
    state = np.reshape(state, [1, num_states])  #reshape for keras
    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    reward_sum += reward
    agent.remember(state[0], action, reward, next_state, done)
    state = next_state
    if done:
        break
env.close()